# Multi Factor Models

In [31]:
import yfinance as yf
import pandas as pd
from scipy import stats
import numpy as np
import plotly.express as px
from scipy.optimize import minimize
import plotly.graph_objects as go
import plotly.io as pio

In [32]:
import pathlib
import sys
utils_path = pathlib.Path().absolute().parent.parent
sys.path.append(utils_path.__str__())
import utils.layout  as lay

In [33]:
pio.templates.default = 'simple_white+blog_mra'

### Portfolio Returns

$$\boldsymbol{y = \alpha + X\beta + \epsilon}$$
$$y = \tilde{X}\tilde{B} + \epsilon $$
$\tilde{X} = (1  X) $

### <font color='Hunter Green'>**Example II.1.3**</font>

In [3]:
betas = np.array([0.8,  1.2])
cov_matrix = np.array([[0.0225, -0.015], [-0.015, 0.04]])
ptf_vol = 0.25


In [4]:
stock_fct_var = betas.T.dot(cov_matrix).dot(betas)
stock_fct_vol = stock_fct_var**0.5
ptf_var = 0.25**2
specs_var = ptf_var - stock_fct_var
specs_vol = specs_var**0.5

In [5]:
print("Volatility due to risk factors: {}".format(stock_fct_vol))
print("Idiosyncratic volatility: {}".format(specs_vol))

Volatility due to risk factors: 0.20784609690826528
Idiosyncratic volatility: 0.13892443989449804


$$ 25\% = (20.78\%^2 + 13.89\%^2) $$

Key points:
* additivity of the variances (not volatilities), but only when we assume zero covariance between specific returns and risk factors
* the portfolio alpha does not affect the risk decomposition, but it does affect the assets expected returns

### Style attribution analysis

### <font color='Hunter Green'>**Example II.1.4**</font>

In [6]:
df = pd.read_excel(r"data\Examples_II.1.xls", sheet_name="EX_II.1.4", usecols="G:O").set_index("Date")
ret = np.log(df/df.shift())[1:]

In [7]:
def min_squared_res(betas0, fund_ret, fct_ret):
    """Minimize squared residuals"""
    return np.sum((fund_ret - fct_ret.dot(betas0))**2)

def sum_weights(betas0):
    return sum(betas0) - 1

def no_short(betas0):
    return betas0

betas0 = [0.5, 0.5, 0.5, 0.5]
cons = [{'type':'eq', 'fun': sum_weights},
         {'type':'ineq', 'fun': no_short}]


In [8]:
#ret_opt = ret.iloc[ret.index<="2004-12-31"]
opt_res = pd.DataFrame(columns=["R1000V", "R1000G", "R2000V", "R2000G"], 
                       index = ["VAA", "VIT", "FAA", "FID"])
for col in ["VAA", "VIT", "FAA", "FID"]:
    res = minimize(min_squared_res, betas0, 
                   args=(ret[col], ret[["R1000V", "R1000G", "R2000V", "R2000G"]]),
                   constraints=cons)
    opt_res.loc[col, :] = res.x

$$\begin{array}{lllll}
\hline
 & R1000V & R1000G & R2000V & R2000G \\
\hline
VAA & 0.95 & 0.02 & -0.00 & 0.02 \\
VIT & 0.93 & -0.00 & 0.00 & 0.07 \\
FAA & 0.35 & 0.05 & 0.00 & 0.59 \\
FID & 0.88 & 0.01 & -0.00 & 0.11 \\
\hline
\end{array}
$$

## Generalization of multifactor models

<font color='Hunter Green'>**Example II.1.5**</font>

In [9]:
betas = np.array([[0.2, 1.2], [0.9, 0.2], [1.3, 0.7]])
weights = np.array([-0.25, 0.75, 0.5])

In [10]:
#net portfolio betas
ptf_net_betas = betas.T.dot(weights)
#using the cov matrix for risk factor of the previous example
ptf_risk_fct_var = ptf_net_betas.dot(cov_matrix).dot(ptf_net_betas)
ptf_risk_fct_vol = ptf_risk_fct_var**0.5
ptf_risk_fct_vol

0.1747185236315829

### Multi-factor models of international portfolios

<font color='Hunter Green'>**Example II.1.6**</font>

In [6]:
data = {'FTSE 100': [2.5,  1.5, 0.20], 'SP 500': [1, 1.2, 0.22], 'DAX 30': [1.5, 0.8, 0.25],
       'USD/GBP': [1,  1, 0.1], 'EUR/GBP': [1.5,  1, 0.12]}
covs = {'FTSE 100': [0.04, 0.035, 0.035, 0.004, 0.0072], 'SP 500': [0.0352, 0.0484, 0.033,-0.0055, 0.00132], 
        'DAX 30': [0.035, 0.033, 0.0625, 0.00125, -0.0045],
       'USD/GBP': [0.004, -0.0055, 0.00125, 0.01, 0.0072], 'EUR/GBP': [0.0072, 0.00132, -0.0045, 0.0072, 0.0144]}
df_ptf = pd.DataFrame(data, index=["Holding", "Beta", "Volatility"])
df_ptf.loc["Weights", : "DAX 30"] = df_ptf.loc["Holding", :"DAX 30"] / df_ptf.loc["Holding", :"DAX 30"].sum()
weights = df_ptf.loc["Weights", : "DAX 30"]
df_covs = pd.DataFrame(covs, index=list(covs.keys()))

$$Portfolio$$
$$\begin{array}{lrrrrr}
\hline
 & FTSE 100 & SP 500 & DAX 30 & USD/GBP & EUR/GBP \\
\hline
Holding & 2.50 & 1.00 & 1.50 & 1.00 & 1.50 \\
Beta & 1.50 & 1.20 & 0.80 & 1.00 & 1.00 \\
Volatility & 0.20 & 0.22 & 0.25 & 0.10 & 0.12 \\
Weights & 0.50 & 0.20 & 0.30 & NaN & NaN \\
\hline
\end{array}$$

$$Covariances$$
$$
\begin{array}{lrrr|rr}
\hline
 & FTSE 100 & SP 500 & DAX 30 &  USD/GBP & EUR/GBP \\
\hline
FTSE 100 & 0.04 & 0.04 & 0.04 & 0.00 & 0.01 \\
SP 500 & 0.04 & 0.05 & 0.03 & -0.01 & 0.00 \\
DAX 30 & 0.04 & 0.03 & 0.06 & 0.00 & -0.00 \\ \hline
USD/GBP & 0.00 & -0.01 & 0.00 & 0.01 & 0.01 \\ 
EUR/GBP & 0.01 & 0.00 & -0.00 & 0.01 & 0.01 \\
\hline
\end{array}
$$

In [42]:
betas = np.diag(df_ptf.loc["Beta", :"DAX 30"])
betas = np.append(betas, [[0, 1, 0], [0,0,1]], axis=0)

$$Betas$$
$$\begin{array}{lrrr}
\hline
 & FTSE 100 & SP 500 & DAX 30 \\
\hline
FTSE 100 & 1.50 & 0.00 & 0.00 \\
SP 500 & 0.00 & 1.20 & 0.00 \\
DAX 30 & 0.00 & 0.00 & 0.80 \\
USD/GBP & 0.00 & 1.00 & 0.00 \\
EUR/GBP & 0.00 & 0.00 & 1.00 \\
\hline
\end{array}$$

In [13]:
bw = betas.dot(weights)

In [15]:
#total variance
sys_var = bw.T.dot(df_covs).dot(bw)
sys_risk = sys_var**0.5

In [18]:
#equity variance
sys_var_equity = bw[:3].T.dot(df_covs.iloc[:3, :3]).dot(bw[:3])
sys_risk_equity = sys_var_equity**0.5

In [19]:
#forex variance
sys_var_forex = bw[3:].T.dot(df_covs.iloc[3:, 3:]).dot(bw[3:])
sys_risk_forex = sys_var_forex**0.5

In [43]:
#quanto variance
sys_var_quanto = bw[:3].T.dot(df_covs.iloc[:3, 3:]).dot(weights[1:].T)
sys_risk_quanto= sys_var_quanto**0.5

In [35]:
labels = ['Equity','Forex','Quanto']
values = [sys_var_equity, sys_var_forex, sys_var_quanto]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_layout(height=600, width=900, title_text="Decomposition of the total systematic variance")
fig.show()